# Analysis of Elevation Profiles of Hiking Trails

## Introduction
This project focuses on the analysis of elevation profiles of hiking trails. The data sources used are hiking trails from OpenStreetMap (OSM) and elevation data from the DMR 5G dataset obtained from the Czech Office for Surveying, Mapping and Cadastre (ČÚZK).

First, the necessary libraries are imported.

In [1]:
# Import standard libraries
import os
import warnings

# Import libraries for data and geospatial data processing
import requests
import geopandas as gpd
import rasterio
from rasterio.plot import show
from shapely.geometry import Point, LineString

# Import libraries for visualization
import matplotlib.pyplot as plt

# Import libraries for working with OSM data
import osmapi

Download of relation using OsmApi

In [2]:
# Function to download a relation by its ID
def download_relation(relation_id):
    try:
        with osmapi.OsmApi() as api:
            return api.RelationGet(relation_id)
    except osmapi.errors.ElementDeletedApiError:
        print(f"Relation with ID {relation_id} has been deleted.")
        return None

Example Usage

In [ ]:
# Example ID
relation_id = 334313
relation = download_relation(relation_id)
print(relation)

In [ ]:
# Function to get coordinates for ways
def get_way_coordinates(way_id):
    try:
        way = osmapi.OsmApi().WayGet(way_id)
        nodes = []
        for node_id in way['nd']:
            node = osmapi.OsmApi().NodeGet(node_id)  # Retrieve the coordinates for each node
            if node:
                nodes.append((node['lon'], node['lat']))  # Collecting the coordinates for the way
        return nodes
    except osmapi.errors.OsmApiError:
        return None

# Example relation ID
relation_id = 222947
relation = download_relation(relation_id)

if relation:
    members = relation.get('member', [])

    # Initialize an empty list for the geometries
    geometries = []

    # Loop through members and filter out only 'way' members
    for member in members:
        if member.get('type') == 'way':
            way_id = member.get('ref')
            way_coords = get_way_coordinates(way_id)
            if way_coords:
                # Add line geometry (Way)
                geometries.append({
                    'type': 'way',
                    'ref': member.get('ref'),
                    'role': member.get('role'),
                    'geometry': LineString(way_coords)  # Store the way as a line
                })

    # Create GeoDataFrame directly from geometries (only ways)
    gdf = gpd.GeoDataFrame(geometries, crs="EPSG:4326")

    # Display the GeoDataFrame on a map
    gdf.plot()
    plt.show()
